In [1]:
from astropy.io import ascii
from astropy.table import vstack, Table, unique, QTable
from astropy.coordinates import SkyCoord 
import astropy.units as u
from astropy import table, log
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord, Distance, Angle
from astropy.time import Time

import os
from IPython.display import display
from multiprocessing import set_start_method

from math import sqrt, cos

In [2]:
# google colab read file in: 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# after mounting drive, copy the path of the directory where the files are stored
save_path='/content/drive/MyDrive/NOFS copy/WiserLabBackup/QueryResults'

In [ ]:
path = '/content/drive/MyDrive/NOFS copy/wdstab6-27.ecsv'
wdstab = Table.read(path, header_start=0, data_start=1) 

In [22]:
# # Read the file

# # vayu's lab comp:
# save_path = 'C:/Users/sc36/Documents/DaphneUSNO/NOFS copy-20230218T215456Z-001/NOFS copy/QueryResults'

# # wiser's lab comp:
# save_path = '/home/student/djz7128/djz_NOFS/QueryResults'

qrt ='{0}/stack_query_results_table.ecsv'.format(save_path) 
ie = '{0}/stack_index_error_queries.ecsv'.format(save_path)
query_results_table = QTable.read(qrt, header_start=0, data_start=1)
index_error_queries = Table.read(ie, header_start=0, data_start=1)


## Assign Units: 

## DEGREES 

deg_columns = ['ra_a', 'ra_b', 'dec_a', 'dec_b']
for deg_column in deg_columns:
    query_results_table[deg_column] = query_results_table[deg_column]*u.deg
    
## MAS

mas_columns = ['ra_error_a', 'ra_error_b', 'dec_error_a', 'dec_error_b', 'parallax_a', 'parallax_b', 'parallax_error_a', 'parallax_error_b']
for mas_column in mas_columns:
    query_results_table[mas_column] = query_results_table[mas_column]*u.mas

## MAS/YR
mas_p_year_columns = ['pmra_a', 'pmra_b', 'pmra_error_a', 'pmra_error_b', 'pmdec_a', 'pmdec_b', 'pmdec_error_a', 'pmdec_error_b']
for mas_p_year_column in mas_p_year_columns:
    query_results_table[mas_p_year_column] = query_results_table[mas_p_year_column]*u.mas/u.yr

In [28]:
duplicates = []
for row in range(len(query_results_table)):
  if row%5000==0:
    print(row)
  row_entry = query_results_table[row]
  for comprow in range(row+1, len(query_results_table)):
    if query_results_table[comprow]==row_entry:
      duplicates.append(comprow)
      continue
    else:
      break


0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000


In [29]:
duplicates

[]

In [26]:
query_results_table.remove_rows(duplicates)

In [43]:
wds_id_error_rows = []
wrong_ids = []

for row in range(len(query_results_table)):

  print(row)
  ra_a, ra_b = query_results_table[row]['ra_a'], query_results_table[row]['ra_b']
  dec_a, dec_b = query_results_table[row]['dec_a'], query_results_table[row]['dec_b']
  id = query_results_table['wds_identifier'][row]
  for comprow in range(row+1, len(query_results_table)):
    compid = query_results_table['wds_identifier'][comprow]
    if compid != id:

      nextid = query_results_table['wds_identifier'][comprow]
      # for nextrow in range(comprow, 3):
        # if query_results_table[nextrow]['wds_identifier'] == nextid:
        #   new_ra_a, new_ra_b = query_results_table[nextrow]['ra_a'], query_results_table[nextrow]['ra_b']
        #   new_dec_a, new_dec_b = query_results_table[nextrow]['dec_a'], query_results_table[nextrow]['dec_b']
        #   if ra_a == new_ra_a and ra_b == new_ra_b and dec_a == new_dec_a and dec_b == new_dec_b:
        #     wds_id_error_rows.append(nextrow)
        #     print(compid)

      if query_results_table[comprow]['wds_identifier'] == nextid:
        new_ra_a, new_ra_b = query_results_table[comprow]['ra_a'], query_results_table[comprow]['ra_b']
        new_dec_a, new_dec_b = query_results_table[comprow]['dec_a'], query_results_table[comprow]['dec_b']
        
        if ra_a == new_ra_a and ra_b == new_ra_b and dec_a == new_dec_a and dec_b == new_dec_b:
          wds_id_error_rows.append(comprow)
          wrong_ids.append(nextid)
          print(compid)
      else:
        break

        
        


0
1
2
3


KeyboardInterrupt: ignored

In [38]:
query_results_table

[]

In [19]:
wdstab[154440]

WDS Identifier,Epoch-1,Epoch-2,Theta-1,Theta-2,Rho-1,Rho-2,Mag-pri,Mag-sec,PMpri-RA,PMpri-DEC,PMsec-RA,PMsec-DEC,"WDS_RA""","WDS_DEC""",RAprideg,DECprideg,RAhms,DECdms,RAsecdeg,DECsecdeg,PMpri-RAdeg,PMpri-DECdeg,PMsec-RAdeg,PMsec-DECdeg,RApri-prepped,DECpri-prepped,RAsec-prepped,DECsec-prepped
,yr,yr,deg,deg,arcsec,arcsec,mag,mag,mas / yr,mas / yr,mas / yr,mas / yr,,,deg,deg,,,deg,deg,deg / yr,deg / yr,deg / yr,deg / yr,deg,deg,deg,deg
str13,int64,int64,int64,int64,float64,float64,str5,str4,float64,float64,float64,float64,str9,str9,float64,float64,str12,str12,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
23592+2740SKF,2003,2015,42,46,1.2,1.0,18.6,20.9,--,--,-10.0,-7.0,235910.07,+274007.7,359.79195833333335,27.668805555555558,23h59m10.07s,+27d40m07.7s,359.7921839503791,27.66899851603133,nan,nan,-2.777777777777778e-06,-1.944444444444445e-06,359.79195833333335,27.668805555555558,359.792133767257,27.66896740491118


In [21]:
wdstab.add_index('WDS Identifier')
id= '23592+4112HJL'
id= '23563+6128STI'
id='08317-1640LDS'
wdsrow=wdstab.loc_indices[id]
wdsrow

54876

In [19]:
wdstab.add_index('WDS Identifier')

mismatch_rows =  []
for row in range(len(query_results_table)):
  if row%5000==0:
    print(row)
  try:
    rra_a = query_results_table[row]['ra_a']/u.deg
    rdec_a = query_results_table[row]['dec_a']/u.deg
    id = query_results_table[row]['wds_identifier']
    wdsrow = wdstab.loc_indices[id]
    wdsra_a = wdstab[wdsrow]['RApri-prepped']
    wdsdec_a = wdstab[wdsrow]['DECpri-prepped']
    wdsra_b = wdstab[wdsrow]['RAsec-prepped']
    wdsdec_b = wdstab[wdsrow]['DECsec-prepped']


    if abs(rra-wdsra_a)>0.002:
      if abs(rra-wdsra_b)>0.002:
        mismatch_rows.append(row)
    elif abs(rdec-wdsdec_a)>0.002:
      if abs(rdec-wdsdec_b)>0.002:
        mismatch_rows.append(row)
  except:
    pass

0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000


In [ ]:
for row in range(len(query_results_table)):
  i

In [20]:
len(mismatch_rows)

90827

In [21]:
len(query_results_table)

107330

In [ ]:
#match catalogs
ra_qrt = query_results_table[row]['ra_a']
c = SkyCoord(ra=ra1*u.degree, dec=dec1*u.degree)
catalog = SkyCoord(ra=ra2*u.degree, dec=dec2*u.degree)
idx, d2d, d3d = c.match_to_catalog_sky(catalog)